In [2]:
import nimfa
from math import sqrt
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
from spherecluster import SphericalKMeans
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import boto3
import sys

In [3]:
def display_estimate(summary, file):
	rss = [summary[rank]['rss'] for rank in rank_cands]
	coph = [summary[rank]['cophenetic'] for rank in rank_cands]
	disp = [summary[rank]['dispersion'] for rank in rank_cands]
	spar = [summary[rank]['sparseness'] for rank in rank_cands]
	spar_w, spar_h = zip(*spar)
	evar = [summary[rank]['evar'] for rank in rank_cands]

	fig = plt.figure(figsize=(14, 10))
	plt.plot(rank_cands, rss, 'o-', label='RSS', linewidth=2)
	plt.plot(rank_cands, coph, 'o-', label='Cophenetic correlation', linewidth=2)
	plt.plot(rank_cands, disp,'o-', label='Dispersion', linewidth=2)
	plt.plot(rank_cands, spar_w, 'o-', label='Sparsity (Basis)', linewidth=2)
	plt.plot(rank_cands, spar_h, 'o-', label='Sparsity (Mixture)', linewidth=2)
	plt.plot(rank_cands, evar, 'o-', label='Explained variance', linewidth=2)
	
	plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, numpoints=1);
	plt.savefig(file, dpi=fig.dpi)

In [4]:
path_file = "matrice-greyscale-64.csv"
estimated_cluster = 128 #40 en temps normal
estimation_nmf = range(125,131,1) #20,200,10

In [5]:
datas = np.loadtxt(open(path_file, "rb"), delimiter=",")
datas = shuffle(datas)
print(datas.shape)
print("Step 2 - Skeams with nb_cluster = ", estimated_cluster)
skm = SphericalKMeans(n_clusters=estimated_cluster)
skm.fit(datas)

(4778, 4096)
('Step 2 - Skeams with nb_cluster = ', 128)


SphericalKMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=128,
        n_init=10, n_jobs=1, normalize=True, random_state=None, tol=0.0001,
        verbose=0)

In [7]:
_data_dimension, _num_bases = datas.shape
W = np.random.random((_data_dimension, estimated_cluster))
H = skm.cluster_centers_

In [8]:
nmf = nimfa.Nmf(datas, seed=None, max_iter=250, H=H, W=W)
nmf_fit = nmf()

W = nmf_fit.basis()
H = nmf_fit.coef()

np.save("step3_W", W)
np.save("step3_H", H)

In [ ]:
sd = "nndsvd"
rank_cands=estimation_nmf
summary = nmf.estimate_rank(rank_range=rank_cands, n_run=1, what='all')
display_estimate(summary, "step3-"+str(m[0])+"-nndsvd.png")